# LFT 12 — Gravity from Constraint Geometry

## Objective
Develop a first toy model of **gravity as curvature of the logical geometry** induced by **constraint density**, consistent with the Logic Field Theory (LFT) program:
- **Matter/Energy** ≙ **dense, persistent logical constraints** in the information space.
- **Gravity** ≙ **distortion of L-flow** and **geodesics** in the emergent state geometry (permutohedral proxy), caused by constraint clusters.

This notebook provides:
1. A discrete **state-space proxy** (2D lattice) for local permutohedral geometry.
2. A **constraint potential** that slows L-flow (time dilation) and warps shortest paths (geodesic bending).
3. A **redshift demo** (two clocks at different constraint densities).
4. A **geodesic demo** (path bending around a “mass” of constraints).

> **Placement in sequence.** Extends LFT_06 (3+1 spacetime) by letting constraint density shape dynamics; complements 10 (Observer) where constraints drive collapse; aligns with 07 (Born) and 11 (Tsirelson) that use the same global Gram/feasibility logic.

## 1) Discrete State-Space Proxy
We use a 2D lattice as a **local chart** of the emergent geometry. Each node is a micro-state; edges represent admissible adjacent transitions of the L-flow.

A **constraint mass** centered at \((x_0,y_0)\) induces a scalar potential \(\Phi\), higher near the mass and decaying with distance. L-flow “speed” is reduced by a factor depending on \(\Phi\).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

nx, ny = 61, 61
x = np.linspace(-3, 3, nx)
y = np.linspace(-3, 3, ny)
X, Y = np.meshgrid(x, y)

# Constraint “mass” parameters
x0, y0 = 0.0, 0.0
alpha = 3.5  # coupling strength of constraints to flow slowdown
r2 = (X - x0)**2 + (Y - y0)**2
Phi = 1.0 / (1.0 + r2)  # simple decaying potential (bounded, smooth)

# L-flow local time-step multiplier: dt_local = 1 + alpha * Phi
dt_local = 1.0 + alpha * Phi

plt.figure(figsize=(6,5))
plt.imshow(Phi, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
plt.colorbar(label='Constraint potential $\Phi$')
plt.title('Constraint density (proxy potential $\Phi$)')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Phi_field.png', dpi=160)
plt.show()

Phi.min(), Phi.max()

## 2) Time Dilation (Redshift) from L-flow Slowdown
A **local clock** advances by one tick per **global step** scaled by the local slowdown factor. If the L-flow is slower near high \(\Phi\), then clocks near the constraint mass tick **fewer** times over the same global step budget — an analogue of **gravitational redshift**.

In [2]:
def simulate_clock_ticks(Phi, alpha=3.5, steps=5000, pos=(0,0)):
    i, j = pos
    dt = 1.0 + alpha * Phi[j, i]
    # Each global step contributes 1/dt local ticks
    return steps / dt

# Select two positions: near mass and far away
near = (nx//2, ny//2)              # center
far = (nx//2, ny//2 + 20)          # displaced along y

ticks_near = simulate_clock_ticks(Phi, alpha=alpha, steps=10000, pos=near)
ticks_far  = simulate_clock_ticks(Phi, alpha=alpha, steps=10000, pos=far)
redshift = ticks_far / ticks_near
ticks_near, ticks_far, redshift

In [3]:
# Visualize ticks per location across the grid to show redshift profile
Ticks = 10000 / (1.0 + alpha * Phi)
plt.figure(figsize=(6,5))
plt.imshow(Ticks, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
plt.colorbar(label='ticks over fixed global steps')
plt.title('Time dilation: slower L-flow near constraint mass')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Redshift_map.png', dpi=160)
plt.show()

## 3) Geodesic Bending via Weighted Paths
Define edge weights as the **local traversal cost** proportional to the slowdown factor, i.e., \(w = 1 + \alpha\,\Phi\). Shortest paths (Dijkstra) then **avoid** high-\(\Phi\) regions, bending around the constraint mass — a discrete analogue of **geodesics in curved spacetime**.

In [4]:
import heapq

W = 1.0 + alpha * Phi  # traversal cost field

def dijkstra_path(W, start, goal):
    ny, nx = W.shape
    INF = 1e18
    dist = np.full((ny,nx), INF)
    prev = np.full((ny,nx,2), -1, dtype=int)
    sx, sy = start; gx, gy = goal
    dist[sy, sx] = 0.0
    pq = [(0.0, sx, sy)]
    while pq:
        d, x0, y0 = heapq.heappop(pq)
        if d>dist[y0,x0]:
            continue
        if (x0,y0)==(gx,gy):
            break
        for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            x1, y1 = x0+dx, y0+dy
            if 0<=x1<nx and 0<=y1<ny:
                nd = d + 0.5*(W[y0,x0] + W[y1,x1])  # midpoint rule cost
                if nd < dist[y1,x1]:
                    dist[y1,x1]=nd
                    prev[y1,x1]=[x0,y0]
                    heapq.heappush(pq, (nd, x1, y1))
    # reconstruct
    path=[]; cur=(gx,gy)
    if prev[gy,gx,0]==-1:
        return []
    while cur!=(sx,sy):
        path.append(cur)
        px,py = prev[cur[1],cur[0]]
        cur=(px,py)
    path.append((sx,sy))
    path.reverse()
    return path, dist[gy,gx]

start = (5, ny//2)
goal  = (nx-6, ny//2)
path, cost = dijkstra_path(W, start, goal)
len(path), cost

In [5]:
# Plot path bending over constraint mass
plt.figure(figsize=(6,5))
plt.imshow(W, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
if path:
    xs = [x[p[0]] for p in path]
    ys = [y[p[1]] for p in path]
    plt.plot(xs, ys, linewidth=2)
plt.title('Geodesic bending around a constraint cluster (higher traversal cost)')
plt.colorbar(label='local traversal cost ~ 1 + αΦ')
plt.tight_layout()
plt.savefig('/mnt/data/LFT_12_Geodesic_bending.png', dpi=160)
plt.show()

## 4) Discussion & Bridge to GR
- **Constraint density** changes the effective metric on the state graph: edges in high-\(\Phi\) regions cost more and clocks tick slower → **time dilation** and **geodesic bending** emerge naturally.
- In the LFT picture, GR’s field equations should arise as **coarse-grained constraints** relating the distribution of \(\Phi\) (from matter/energy) to an effective metric on the emergent manifold.
- Next steps:
  1. Replace the lattice with **local permutohedral charts** from 02.
  2. Calibrate slowdown rule vs. **inversion-count dynamics** (03) to derive a specific functional form.
  3. Fit an **effective metric tensor** and test for analogs of redshift and lensing vs. GR predictions.
  4. Explore **constraint conservation laws** → analogs of stress-energy conservation.

### Artifacts
- `/mnt/data/LFT_12_Phi_field.png`
- `/mnt/data/LFT_12_Redshift_map.png`
- `/mnt/data/LFT_12_Geodesic_bending.png`

## 10) Deflection angle vs. impact parameter (with GR comparison)
We estimate the bending angle by tracing rays with different impact parameters and computing the angle between initial and final tangents. For a weak, spherically symmetric lens, GR predicts
$$ \Delta\theta_{\rm GR}(b) \approx \frac{4 G M}{c^2 b}. $$
Because our \(\Phi\) is defined up to scale, we **fit a single scale factor** so the curves can be compared on shape (\(\propto 1/b\)).

In [10]:
import numpy as np, matplotlib.pyplot as plt
try:
    n_eff
except NameError:
    # Recreate a minimal setup if this cell is run standalone
    nx, ny = 161, 161
    x = np.linspace(-5,5,nx); y = np.linspace(-5,5,ny)
    X,Y = np.meshgrid(x,y)
    sigma = 0.5
    rho_c = np.exp(-((X**2+Y**2)/(2*sigma**2)))
    rho_c /= rho_c.sum()
    def solve_poisson_periodic(rhs, Lx, Ly):
        ny, nx = rhs.shape
        kx = np.fft.fftfreq(nx, d=Lx/nx) * 2*np.pi
        ky = np.fft.fftfreq(ny, d=Ly/ny) * 2*np.pi
        KX, KY = np.meshgrid(kx, ky); K2 = KX**2 + KY**2
        rhs_k = np.fft.fft2(rhs)
        Phi_k = np.zeros_like(rhs_k, dtype=complex); mask = K2!=0
        Phi_k[mask] = - rhs_k[mask] / K2[mask]
        Phi = np.fft.ifft2(Phi_k).real
        return Phi
    Phi = solve_poisson_periodic(rho_c, Lx=x.max()-x.min(), Ly=y.max()-y.min())
    epsilon = 0.02; gamma_ppn = 1.0
    n_eff = 1.0 + (1.0+gamma_ppn)*epsilon*Phi
    from scipy.ndimage import sobel
    gx = sobel(n_eff, axis=1)/(x[1]-x[0])/8.0
    gy = sobel(n_eff, axis=0)/(y[1]-y[0])/8.0

from scipy.ndimage import map_coordinates

def trace_ray(x0, y0, vx=1.0, vy=0.0, dt=0.01, steps=6000):
    xs=[x0]; ys=[y0]; v=np.array([vx,vy], float)
    pos=np.array([x0,y0], float)
    for _ in range(steps):
        ix = (pos[0]-x.min())/(x.max()-x.min())*(nx-1)
        iy = (pos[1]-y.min())/(y.max()-y.min())*(ny-1)
        if ix<1 or ix>nx-2 or iy<1 or iy>ny-2:
            break
        gx_val = map_coordinates(gx, [[iy],[ix]], order=1, mode='nearest')[0]
        gy_val = map_coordinates(gy, [[iy],[ix]], order=1, mode='nearest')[0]
        a = -np.array([gx_val, gy_val])
        v = v + a*dt
        v = v/np.linalg.norm(v)
        pos = pos + v*dt
        xs.append(pos[0]); ys.append(pos[1])
    return np.array(xs), np.array(ys)

def deflection_angle(xs, ys):
    # estimate tangent at start and end
    if len(xs)<5:
        return np.nan
    v0 = np.array([xs[3]-xs[0], ys[3]-ys[0]])
    v1 = np.array([xs[-1]-xs[-4], ys[-1]-ys[-4]])
    v0 = v0/np.linalg.norm(v0); v1 = v1/np.linalg.norm(v1)
    dot = np.clip(np.dot(v0, v1), -1.0, 1.0)
    ang = np.arccos(dot)
    return ang

impacts = np.linspace(0.5, 2.5, 9)
b_vals=[]; th_vals=[]
for b in impacts:
    xs, ys = trace_ray(x.min()+0.5, b, vx=1.0, vy=0.0, dt=0.01, steps=8000)
    th = deflection_angle(xs, ys)
    b_vals.append(b); th_vals.append(th)
b_vals = np.array(b_vals); th_vals = np.array(th_vals)

# Fit GR shape A/b to the simulated deflections
mask = np.isfinite(th_vals) & (th_vals>0)
A_fit = np.sum(th_vals[mask]*b_vals[mask]**-1)/np.sum((b_vals[mask]**-1)**2)
theta_gr = A_fit / b_vals

plt.figure(figsize=(6,4))
plt.plot(b_vals, th_vals, marker='o', label='Simulated Δθ(b)')
plt.plot(b_vals, theta_gr, marker='x', label='Scaled GR ~ A/b')
plt.xlabel('impact parameter b')
plt.ylabel('deflection angle Δθ (radians)')
plt.title('Deflection vs impact parameter: model vs 1/b law')
plt.legend(); plt.tight_layout(); plt.savefig('/mnt/data/LFT_12_Deflection_vs_b.png', dpi=160); plt.show()
A_fit

## 11) Poisson solver with open (Dirichlet) boundaries
To avoid periodic-image artifacts, we add a finite-difference Poisson solver with **Dirichlet boundary** \(\Phi=0\) at the domain edge. This approximates an **open** boundary for localized sources.

In [11]:
import numpy as np, matplotlib.pyplot as plt

def solve_poisson_dirichlet(rhs, dx, dy, iters=2000, omega=1.8):
    # Solve ∇^2 Phi = rhs on a rectangle with Phi=0 at boundaries via SOR
    ny, nx = rhs.shape
    Phi = np.zeros_like(rhs)
    ax = 1.0/dx**2; ay = 1.0/dy**2; denom = 2*(ax+ay)
    for _ in range(iters):
        for j in range(1,ny-1):
            for i in range(1,nx-1):
                Phi_new = (ax*(Phi[j, i-1]+Phi[j, i+1]) + ay*(Phi[j-1, i]+Phi[j+1, i]) - rhs[j,i]) / denom
                Phi[j,i] = (1-omega)*Phi[j,i] + omega*Phi_new
    return Phi

# Build a localized source and solve
nx2, ny2 = 121, 121
x2 = np.linspace(-5,5,nx2); y2 = np.linspace(-5,5,ny2)
X2, Y2 = np.meshgrid(x2, y2)
sigma2 = 0.6
rhs = np.exp(-((X2**2+Y2**2)/(2*sigma2**2)))
rhs /= rhs.sum()
PhiD = solve_poisson_dirichlet(rhs, dx=x2[1]-x2[0], dy=y2[1]-y2[0], iters=1500, omega=1.7)

plt.figure(figsize=(6,5))
plt.imshow(PhiD, origin='lower', extent=[x2.min(),x2.max(),y2.min(),y2.max()])
plt.colorbar(label='Φ (Dirichlet Poisson)')
plt.title('Constraint potential with open boundaries (Dirichlet)')
plt.tight_layout(); plt.savefig('/mnt/data/LFT_12_Phi_dirichlet.png', dpi=160); plt.show()